### Import Libraries

In [ ]:

import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom


import os
import matplotlib.pyplot as plt
import keras

import cv2

In [ ]:
df=pd.read_csv('/XSS_dataset.csv', encoding='utf-8-sig')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head()

In [ ]:
df=df[df.columns[-2:]]  # Only get sentence and labels

In [ ]:
df.head()

In [ ]:
# Get Sentences data from data frame
sentences=df['Sentence'].values
sentences[1]

In [ ]:
print(len(sentences))

In [ ]:
def convert_to_ascii(sentence):
    sentence_ascii = []
    for i in sentence:
        if ord(i) < 8222:  # Adjusted condition
            if ord(i) == 8221:  # ” : 8221
                sentence_ascii.append(129)
            elif ord(i) == 8220:  # “ : 8220
                sentence_ascii.append(130)
            elif ord(i) == 8216:  # ‘ : 8216
                sentence_ascii.append(131)
            elif ord(i) == 8217:  # ’ : 8217
                sentence_ascii.append(132)
            elif ord(i) == 8211:  # – : 8211
                sentence_ascii.append(133)
            else:
                sentence_ascii.append(ord(i))
    zer = np.zeros((10000))
    for i in range(len(sentence_ascii)):
        zer[i] = sentence_ascii[i]
    zer.shape = (100, 100)
    return zer


In [ ]:


# send each sentence to be converted to ASCII


arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):

    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128


#     if i==1:
#         plt.plot(image)
#         plt.show()
    arr[i]=image



In [ ]:
print("Input data shape : ", arr.shape)

In [ ]:
# Reshape data for input to CNN
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [ ]:
data.shape

In [ ]:
y=df['Label'].values

In [ ]:
# Split into train/test data
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)
trainX, validX, trainY, validY = train_test_split(trainX,trainY, test_size=0.2, random_state=42)

In [ ]:
# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Bidirectional

# Define the model
model = Sequential([
    Bidirectional(SimpleRNN(64, activation='relu', return_sequences=True), input_shape=(100, 100)),

    # Add dropout for regularization if needed
    # tf.keras.layers.Dropout(0.2),

    Bidirectional(SimpleRNN(128, activation='relu', return_sequences=True)),
    Bidirectional(SimpleRNN(256, activation='relu', return_sequences=True)),

    # Add more SimpleRNN layers if needed

    # Flatten the output of the RNN layers
    tf.keras.layers.Flatten(),

    # Dense layers
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])



In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
# Reshape input data to have 3 dimensions
trainX_reshaped = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2])

# Define batch size and number of epochs
batch_size = 128
num_epochs = 10

validX_reshaped = validX.reshape(validX.shape[0], validX.shape[1], validX.shape[2])

# Model training
model_log = model.fit(trainX_reshaped, trainY,
                      batch_size=batch_size,
                      epochs=num_epochs,
                      verbose=1,
                      validation_data=(validX_reshaped, validY))

Epoch 1/10
69/69 [==============================] - 143s 2s/step - loss: 0.2417 - accuracy: 0.8939 - val_loss: 0.1122 - val_accuracy: 0.9662
Epoch 2/10
69/69 [==============================] - 126s 2s/step - loss: 0.0912 - accuracy: 0.9719 - val_loss: 0.0630 - val_accuracy: 0.9813
Epoch 3/10
69/69 [==============================] - 128s 2s/step - loss: 0.0676 - accuracy: 0.9784 - val_loss: 0.0659 - val_accuracy: 0.9813
Epoch 4/10
69/69 [==============================] - 132s 2s/step - loss: 0.0460 - accuracy: 0.9860 - val_loss: 0.0594 - val_accuracy: 0.9840
Epoch 5/10
69/69 [==============================] - 143s 2s/step - loss: 0.0373 - accuracy: 0.9880 - val_loss: 0.0541 - val_accuracy: 0.9849
Epoch 6/10
69/69 [==============================] - 148s 2s/step - loss: 0.0322 - accuracy: 0.9902 - val_loss: 0.0549 - val_accuracy: 0.9831
Epoch 7/10
69/69 [==============================] - 133s 2s/step - loss: 0.0491 - accuracy: 0.9832 - val_loss: 0.0632 - val_accuracy: 0.9831
Epoch 8/10
69

In [ ]:
# Plot model architecture
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='rnn_model.png', show_shapes=True)

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
plot_hist(model_log)

### Confusion Matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have already trained your model and obtained predictions
predictions = model.predict(validX_reshaped)

# Convert predictions to binary labels (0 or 1)
binary_predictions = np.round(predictions)

# Get the confusion matrix
conf_matrix = confusion_matrix(validY, binary_predictions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks([0, 1], ['Negative', 'Positive'])
plt.yticks([0, 1], ['Negative', 'Positive'])

# Add labels to each cell
thresh = conf_matrix.max() / 2.
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

plt.show()

# Generate classification report
class_report = classification_report(validY, binary_predictions, target_names=['Negative', 'Positive'])
print("Classification Report:")
print(class_report)


In [ ]:
# Save the model
model.save('rnn_model.h5')
loaded_model = tf.keras.models.load_model('rnn_model.h5')

In [ ]:
def convert_to_ascii(sentence):
    sentence_ascii = []
    for i in sentence:
        if ord(i) < 8222:  # Adjusted condition
            if ord(i) == 8221:  # ” : 8221
                sentence_ascii.append(129)
            elif ord(i) == 8220:  # “ : 8220
                sentence_ascii.append(130)
            elif ord(i) == 8216:  # ‘ : 8216
                sentence_ascii.append(131)
            elif ord(i) == 8217:  # ’ : 8217
                sentence_ascii.append(132)
            elif ord(i) == 8211:  # – : 8211
                sentence_ascii.append(133)
            else:
                sentence_ascii.append(ord(i))
    zer = np.zeros((10000))
    for i in range(len(sentence_ascii)):
        zer[i] = sentence_ascii[i]
    zer.shape = (100, 100)
    return zer


def predict(sentence):
    image=convert_to_ascii(sentences[i])
    x=np.asarray(image,dtype='float')
    vec =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    vec/=128
    reshaped_vector = vec.reshape(1, 100, 100)
    return 1 if model.predict(reshaped_vector)[0][0] >= 0.5 else 0

In [ ]:
df_ = pd.read_csv("XSS_dataset.csv")

df_.head()

In [ ]:
correct = 0
incorrect = 0
from tqdm import tqdm

for i in tqdm(range(len(df_))):
    sentence = df_.iloc[i]["Sentence"]
    label = df_.iloc[i]["Label"]
    prediction = 0
    if predict(sentence)[0][0] >= 0.5:
        prediction = 1
    if prediction==label:
        correct+=1
    else:
        incorrect+=1


print(correct)
print(incorrect)